<a href="https://colab.research.google.com/github/giovannabbottino/SBD2/blob/main/Bancos_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
import os
import datetime
from tqdm import tqdm
import zipfile

In [ ]:
class Gerador:
  tipo_script = ''

  def __init__(self, base_dados, autores, tabelas, alteracoes):
    '''
    base_dados = "nome_base"
    autores = [autor1, autor2]
    tabelas = [tabela1, tabela2]
    alreacoes = {data: alteracao}
    '''
    self.base_dados = base_dados
    self.autores = autores
    self.tabelas = tabelas
    self.alteracoes = alteracoes

  def cabecalho(self):
    self.script = '-- ------        TRABALHO FINAL ---------- --\n\
--                                                      --\n\
--                    SCRIPT DE %s           --\n\
--                                                      --\n\
-- Data Criacao ...........: %s--\n\
-- Autor(es) ..............: %s--\n\
-- Banco de Dados .........: MySQL                      --\n\
-- Base de Dados (nome) ...: %s--\n\
--                                                      --\n\
-- Ultimas Alterações                                   --\n\
-- 		%s 		--\n\
--                                                      --\n\
-- PROJETO => 1 Base de Dados                           --\n\
--         => %s Tabelas                                 --\n\
--                                                      --\n\
-- ---------------------------------------------------- --\n\
-- BASE DE DADOS\n\
%s\n\
\n\
USE %s;\n\
\n\
-- TABELAS\n' % (
    self.tipo_script, 
    datetime.datetime.now().strftime("%d/%m/%Y"),
    '--\n-- 			'.join(self.autores),
    self.base_dados,
    (((((str(self.alteracoes)).replace(':', '=>')).replace(',', '--\n-- 		')).replace('{','')).replace('}','')).replace("'",''),
    str(len(self.tabelas)), 
    'CREATE DATABASE\nIF NOT EXISTS %s;\n' % (self.base_dados) if self.tipo_script == 'CRIACAO (DDL)' else '',
    self.base_dados)

  def salva(self, arquivo):
    f = open(arquivo, "a")
    f.write(self.script)
    f.close()

  def salva_le(self, arquivo):
    f = open(arquivo, "w")
    f.write(self.script)
    f.close()

    f = open(arquivo, "r")
    print(f.read())

In [ ]:
class Fisico(Gerador):
  tipo_script = 'CRIACAO (DDL)'

  def insere(self, df, nome_tabela):
    self.script += '\nCREATE TABLE %s (\n' % (nome_tabela.replace(" ", "_"))

    for i, type_ in enumerate(df.dtypes):      
      self.script += '  %s %s NOT NULL,\n' % (
          (df.columns.tolist())[i], 
          ((str(type_).replace("float64", "FLOAT")).replace("int64", "INT")).replace("object", "VARCHAR(20)"))
      
    self.script = self.script[:-2] + ') ENGINE InnoDB;\n'

In [ ]:
class Apaga(Gerador):
  tipo_script = 'APAGA (DML)  '

  def insere(self):
    self.script += 'DROP TABLE %s;\n' % (';\nDROP TABLE '.join(self.tabelas))

In [ ]:
class Popula(Gerador):
  tipo_script = 'POPULAR (DML)'

  def insere(self, df, nome_tabela, arquivo):
    self.script += '\nINSERT INTO  %s (%s) VALUES\n' % (nome_tabela.replace(" ", "_"), ', '.join(df.columns.tolist()))

    for index, row in tqdm(df.iterrows(), total=df[df.columns[0]].count()):
      self.script += '('
      for i, r in enumerate(row):
        if isinstance(r, str):
          r = "'%s'" % (r)
        if r == "NaN" or r == None or r == "nan":
          r = 'NULL'

        r = str(r)
        self.script += r + ', '

      self.script += r + '),\n'
      self.salva_popula(arquivo)
    self.script = self.script[:-2] + ';\n'

  def salva_popula(self, arquivo):
    f = open(arquivo, "w")
    f.write(self.script)
    f.close()


In [ ]:
class MER(Gerador):
  index = 0
  def __init__(self, tabelas):
    '''
    tabelas = [tabela1, tabela2]
    '''
    self.tabelas = tabelas

  def cabecalho(self):
    self.script = 'Modelo Entidade-Relacionamento (ME-R)\n\
Entidades\n\
%s\n\
Atributos\n' % ('\n'.join(self.tabelas) )

  def insere(self, df):
    self.script += '%s (%s)\n' % (self.tabelas[self.index], ', '.join([c for c in df.columns.tolist()]))
    self.index += 1

In [ ]:
a_directory = "tabelas"

base_dados = 'teste'
autores = ['Stefano Leone', 'Giovanna Borges Bottino', 'Victor Amaral Cerqueira']

tabelas = [(name.replace(".csv", "")).replace(" ", "_") for name in os.listdir(a_directory)]

alteracoes = {datetime.datetime.now().strftime("%d/%m/%Y"): 'Transforma csv em script'}
fisico = Fisico(base_dados, autores, tabelas, alteracoes)
fisico.cabecalho()

alteracoes = {datetime.datetime.now().strftime("%d/%m/%Y"):'Adiciona drop das tabelas' }
apaga = Apaga(base_dados, autores, tabelas, alteracoes)
apaga.cabecalho()
apaga.insere()
apaga.salva_le('apaga.sql')

alteracoes = {datetime.datetime.now().strftime("%d/%m/%Y"):'Popula tabelas' }
popula = Popula(base_dados, autores, tabelas, alteracoes)
popula.cabecalho()

mer = MER(tabelas)
mer.cabecalho()

for i, filename in tqdm(os.listdir(a_directory)):
  filepath = os.path.join(a_directory, filename)

  df = pandas.read_csv(filepath, engine='python', encoding='utf-8', error_bad_lines=False)
  popula.insere(df, filename.replace(".csv", ""), 'popula.sql')
  fisico.insere(df, filename.replace(".csv", ""))
  mer.insere(df)

fisico.salva_le('fisico.sql')
popula.salva_le('popula.sql')

mer.salva_le('mer.txt')

In [ ]:
import pandas as pd

filename = '/content/IMDb names.csv'
df = pd.read_csv(filename, nrows=180000)

df.to_csv(filename)